In [2]:
from utils.train import get_Visdom
from utils.load_raw_yelp import *
from PIL import Image, ImageDraw, ImageFont
from models.lymodel5 import *
from utils.tokenization import BasicTokenizer
viz = get_Visdom()

Setting up a new session...


In [3]:
PATH = "mymodel.pth"
*_, glove_data = load_glove_data(config)
model = Model(config)

/home/ly/miniconda3/envs/torch1.7/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [4]:
data = load_clean_data()["test"]
collte_fn = get_collate_fn(config)
glove_vocab = load_glove_vocab(DATA_DIR + "raw/")
glove_tokenizer = YelpSimpleTokenizer(glove_vocab["token2idx"], True)
def load_image(_id):
    path = DATA_DIR + "photos/" + _id[:2] +"/" + _id + ".jpg"
    return Image.open(path)
def get_data(idx, city="Boston"):
    return glove_tokenizer.tokenize(data[city][idx]["Text"]), data[city][idx]["Photos"], data[city][idx]["Rating"]\
        , collte_fn([glove_data[city][idx]])

In [4]:
text, photos, rating, (X, y) = get_data(0)

In [5]:
state_dict = torch.load(PATH)
model.load_state_dict(state_dict)
model.eval()

Model(
  (word_embedding): Embedding(42183, 100)
  (word_encoder): DynamicRNN(
    (RNN): GRU(100, 100, batch_first=True, dropout=0.1, bidirectional=True)
  )
  (img_fc): Linear(in_features=2048, out_features=200, bias=True)
  (tanh1): Tanh()
  (attn): MultiheadAttention(
    (out_proj): _LinearWithBias(in_features=200, out_features=200, bias=True)
  )
  (forget_gate_img_1): Linear(in_features=200, out_features=200, bias=False)
  (forget_gete_text_1): Linear(in_features=200, out_features=200, bias=True)
  (sigmoid): Sigmoid()
  (forget_gate_img_2): Linear(in_features=200, out_features=200, bias=False)
  (forget_gete_text_2): Linear(in_features=200, out_features=200, bias=True)
  (tanh2): Tanh()
  (fusion_encoder): MultiheadAttentionEncoder(
    (attn): MultiheadAttention(
      (out_proj): _LinearWithBias(in_features=200, out_features=200, bias=True)
    )
  )
  (output_layer): OutputLayer(
    (fc): Linear(in_features=200, out_features=5, bias=True)
    (dropout): Dropout(p=0.1, inpla

In [42]:
font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", 40, encoding="unic")#设置字
draw.text((100, 50), u'Hello World', 'fuchsia', font)

In [6]:
def get_bbox(npz):
    val = np.load(npz)
    bbox = val["bbox"].astype(np.int32).tolist()
    return bbox
def _draw_bbox(im, bbox):
    draw = ImageDraw.Draw(im)
    for box in bbox:
        draw.rectangle(box, None, 'red')
    return im

In [7]:
def draw_photo_bbox(_id, indicies=None): # 显示图和bbox
    im = load_image(_id)
    bbox = load_rcnn_features(_id)["bbox"].astype(np.int32).tolist()
    draw = ImageDraw.Draw(im)
    if indicies:
        for i in indicies:
            draw.rectangle(bbox[i], None, 'red')
    else:
        for box in bbox:
            draw.rectangle(box, None, 'red')
    im = np.array(im).transpose(2, 0, 1)
    win = viz.image(im)

In [8]:
def look_yelp_data(text, rating, photos):
    print("Text:")
    print(text)
    print(f"Rating : {rating}")
    for i in photos:
        draw_photo_bbox(i)

In [9]:
def get_data2(idx, city="Boston"):
    return data[city][idx]["Text"], data[city][idx]["Photos"], data[city][idx]["Rating"]\
        , collte_fn([glove_data[city][idx]])

In [ ]:
text, photos, rating, (X, y) = get_data2(2)
look_yelp_data(text, rating, photos)

In [10]:
def tok_1d(text):
    sents = glove_tokenizer.tokenize(text)
    res = []
    for s in sents:
        res += s
    return res

In [12]:
def get_all(sents, word):
    res = []
    for i, s in enumerate(sents):
        if s == word:
            res.append(i)
    return res
# indices = get_all(tok_1d(text), "sushi")
# indices

In [13]:
def get_bbox_num(photos):
    res = []
    for _id in photos:
#         print(_id)
        bbox = load_rcnn_features(_id)["bbox"].astype(np.int32).tolist()
        res.append(len(bbox))
    return res
# box_num = get_bbox_num(photos)

In [65]:
res, score = model.infer(X)

In [66]:
score.shape

torch.Size([1, 265, 66])

In [68]:
sushi_score = score[0][indices]

In [99]:
val, indices = sushi_score[4].topk(3)

In [14]:
def idx2box_num(indices, box_num):
    res = []
    for i in indices:
        for j, n in enumerate(box_num):
            if i >= n:
                i -= n
            else:
                res.append((j, i))
    return res


In [115]:
indices

tensor([11,  8,  9])

In [103]:
box_num

[16, 18, 16, 16]

In [15]:
def draw_photo_bbox(_id, indicies=None, color='green', width=1): # 显示图和bbox
    im = load_image(_id)
    bbox = load_rcnn_features(_id)["bbox"].astype(np.int32).tolist()
    draw = ImageDraw.Draw(im)
    if indicies:
        for i in indicies:
            draw.rectangle(bbox[i], None, color, width)
    else:
        for box in bbox:
            draw.rectangle(box, None, color, width)
    im = np.array(im).transpose(2, 0, 1)
    win = viz.image(im)

In [116]:
val, indices = sushi_score[3].topk(3)
_te = idx2box_num(indices, box_num)
_te

[(1, tensor(11)),
 (2, tensor(11)),
 (3, tensor(11)),
 (1, tensor(8)),
 (2, tensor(8)),
 (3, tensor(8)),
 (3, tensor(9))]

In [125]:
text

"despite of 3 and 1/2 stars , i gave it a try .|||should have left the moment i heard the conversations among the `` chefs '' at the sushi bar , the moment when i saw the `` extensive '' menu and when i learned that it is a restaurant chain .|||if i had my dinner at one of its tables , i might have given it a two-star rating .|||( my one star rating is for the waitress who served me ) sushi bar is where the chefs shine .|||have been at most of the top notch japanese restaurants in nyc , it is a blessing and a curse !|||can you imagine a neurosurgeon with gloved hands , touches your brain with an open skull , touches the breathing tubes , shakes hands with the other surgeons , opens and holds the operating room doors for the staffs and keeps on flipping the paper charts ?|||and if i am not too old fashion about sushi , it is it 's complexity from such simple ingredients ( meaning sushi rice and fish ) that makes sushi great .|||not the sauce !|||despite of how good my dinner looks in pi

In [127]:
draw_photo_bbox(photos[3], [11], color="yellow", width=5)

In [120]:
im = load_image(photos[3])
draw = ImageDraw.Draw(im)

In [149]:
text, photos, rating, (X, y) = get_data2(9)
look_yelp_data(text, rating, photos)

Text:
the tried their bacon donut again and it did not live up to the hype !|||the donut itself was very dry and flakey ?|||i ca n't describe it .|||it leaves an oily taste in my mouth .|||i would give it another try .|||hoping the 3rd time is different .
Rating : 2


In [150]:
res, score = model.infer(X)
score.shape

torch.Size([1, 56, 48])

In [151]:
indices = get_all(tok_1d(text), "bacon")
indices

[3]

In [152]:
score[0][3].topk()

torch.return_types.topk(
values=tensor([0.0220, 0.0219, 0.0219, 0.0218, 0.0216], grad_fn=<TopkBackward>),
indices=tensor([ 3, 21, 12,  7, 44]))

In [153]:
box_num = get_bbox_num(photos)
box_num

[16, 16, 16]

In [154]:
draw_photo_bbox(photos[0], [3], color="yellow", width=5)

In [16]:
def idx2box_num_v2(indices, box_num):
    res = []
    for i in indices:
        for j, n in enumerate(box_num):
            if i >= n:
                i -= n
            else:
                res.append((j, i))
                break
    return res

## 4

In [197]:
text, photos, rating, (X, y) = get_data2(24)
res, score = model.infer(X)
look_yelp_data(text, rating, photos)

Text:
went here for restaurant week march 2016 for lunch .|||staff were tired and inattentive .|||food was n't very filling ( even though one of our waiters gave us an extra cream spinach by accident ) .|||we asked a waiter for bread , he walked away and never came back .|||had to ask a waitress for bread again and i asked her for butter , she came back again without the butter .|||i know it all seems so picky but i do n't think i was asking for a lot .|||the steak was good but considering the size of it i ca n't imagine paying the normal $ 43 for it .|||the mashed potatoes and cream spinach were good too , however the quality of the food did n't make up for the quality of the service .|||considering all these other positive reviews i hope it was just an off day , because i 've had better service at tgi fridays .
Rating : 2


In [198]:
indices = get_all(tok_1d(text), "steak")
box_num = get_bbox_num(photos)
_, bbox_idx = score[0][indices[0]].topk(5)
temp = idx2box_num_v2(bbox_idx, box_num)
for i, j in temp:
    draw_photo_bbox(photos[i], [j.item()], color="yellow", width=5)

In [162]:
draw_photo_bbox(photos[1], [4], color="yellow", width=5)

## 5

In [27]:
text, photos, rating, (X, y) = get_data2(32)
res, score = model.infer(X)
look_yelp_data(text, rating, photos)

Text:
this place has really good chicken wings , but i must say its not worth the wait/ horrible service they have .|||ok maybeeee it is !|||one time , i made my order at 5:00pm , ( considering we all know they take about 40+ minutes to get a order ready ) i stopped by the place at about 5:40ish to pick up the order -- - the waiter said `` i 'm sorry , it will be another 15 minutes '' .|||so my boyfriend and i decided to walk around the area and came back around 6:00ish , just so the waiter can spend another 5 minutes looking for my order in his computer , then telling me he didnt put the order in , long story short ; it took 1hr and 15 mins for take out wings .|||the environment is pretty chill , its not too fancy .|||the place is n't too big and do n't be surprised if you have to wait a few minutes for a table ; the place can really get crowded !
Rating : 4


In [31]:
indices = get_all(tok_1d(text), "wings")
box_num = get_bbox_num(photos)
_, bbox_idx = score[0][indices[0]].topk(3)
temp = idx2box_num_v2(bbox_idx, box_num)
for i, j in temp:
    draw_photo_bbox(photos[i], [j.item()], color="yellow", width=5)

## 6

In [41]:
text, photos, rating, (X, y) = get_data2(37)
res, score = model.infer(X)
look_yelp_data(text, rating, photos)

Text:
i am known to travel far and wide for a great coffee shop on the weekends .|||would i go for a 20 minute drive to come back here , probably not .|||i would definitely hit it up again if i was close by .|||the decor of this place suits it 's name , it has a cool nostalgic american feel to it .|||iced mocha - standard iced mocha , was up to par with other good coffee shops i have been to but probably would n't crack my top 3 .|||cheese danish - solid cheese danish , ca n't go wrong on this .
Rating : 4


In [40]:
indices = get_all(tok_1d(text), "coffee")
box_num = get_bbox_num(photos)
_, bbox_idx = score[0][indices[0]].topk(3)
temp = idx2box_num_v2(bbox_idx, box_num)
for i, j in temp:
    draw_photo_bbox(photos[i], [j.item()], color="yellow", width=5)

In [ ]:
burger 